In [ ]:
from __future__ import print_function
import getpass
import os
import sys
from subprocess import Popen, check_output
from sys import stderr
from time import sleep
import io
import matplotlib.pyplot as plt
import pandas as pd
import tempfile
import subprocess
import psutil

In [ ]:
def FetchCPUProcesses(mostImportant=10):
    procs = []
    for proc in psutil.process_iter():
        if proc.pid == os.getpid(): continue 
        if getpass.getuser() in proc.username():
            procs.append({'pid':            proc.pid,
                          'cpu_percent':    proc.cpu_percent(),
                     for proc in psutil.process_iter():
    if proc.pid == os.getpid(): continue 
    if proc.status() != 'running': continue
    if getpass.getuser() == proc.username():
        procs.append({'pid':            proc.pid,
                      'cpu_percent':    proc.cpu_percent(),
                      'memory_percent': proc.memory_percent(),
                      'name':           proc.name(),
                      'exe':            proc.exe(),
                      'status':         proc.status()
                     })

process_log = pd.DataFrame(procs)     'memory_percent': proc.memory_percent(),
                          'name':           proc.name(),
                          'exe':            proc.exe(),
                          'status':         proc.status()
                         })
    process_log = pd.DataFrame(procs)
    tmp = process_log.sort_values(['memory_percent'], ascending=False)[:10]
    return tmp

In [ ]:
def GetProcessAttributes(pids):
    processAttributes = []
    for pid in pids:
        proc = psutil.Process(pid)
        processAttributes.append({'pid':     pid,
                                  'name':    proc.name(),                
                                  'cmdline': proc.cmdline(),
                                  'status':  proc.status()})
    processAttributes = pd.DataFrame(processAttributes)
    return processAttributes

In [ ]:
def MonitorNvidiaGPU():
    'Function that monitors Running Processes on Nvidia GPU'
    '''
        Returns a DataFrame (pid, process_name, cmdline, used_gpu_memory, utilization)
    '''
    getGPUProcesses = 'nvidia-smi pmon -c 1 -s mu'
    
    proc = subprocess.Popen(getGPUProcesses, shell=True, stdout=subprocess.PIPE)
    output = proc.stdout.read().decode('utf-8').split('\n')
    
    # Remove the line describing the units of each feature
    del output[1]
    # convert to csv format...
    output[0] = output[0].replace('# ', '')
    output = [line.strip() for line in output]
    output = [','.join(line.split()) for line in output]
    # ...and drop the command feature (will be added later)...
    output = [','.join(line.split(',')[:8]) for line in output]
    # ...and convert to DataFrame
    procsGPU = pd.read_csv(io.StringIO('\n'.join(output)), header=0)
    procsGPUFeats = GetProcessAttributes(procsGPU.pid.values)
    
    return procsGPU.merge(procsGPUFeats, on='pid', how='inner')

In [ ]:
def full_outer_join(df_x, df_y, on=None, left_on=None, right_on=None):
    df = pd.merge(df_x, df_y, on='pid', how='outer', suffixes=('', '_y'))
    # list comprehension of the cols that end with '_y'
    to_drop = [x for x in df if x.endswith('_y')]
    df.drop(to_drop, axis=1, inplace=True)
    return df

In [ ]:
%timeit MonitorNvidiaGPU()

### Notes:
* ### Decide if we use the default Nvidia thread or our own (So Far: our own)

In [ ]:
#!/usr/bin/python
from subprocess import Popen, PIPE, STDOUT
import pty
import os

cmd = 'nvidia-smi pmon -c 1 -s mu'

proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE)
output = proc.stdout.read().decode('utf-8').split('\n')
output

In [ ]:
procs = []
for proc in psutil.process_iter():
    if proc.pid == os.getpid(): continue 
    
    try:
        if getpass.getuser() == proc.username():
            procs.append({'pid':            proc.pid,
                          'cpu_percent':    proc.cpu_percent(),
                          'memory_percent': proc.memory_percent(),
                          'name':           proc.name(),
                          'exe':            proc.exe(),
                          'user':           proc.username(),
                          'status':         proc.status()
                         })
    except psutil.AccessDenied:
        continue
    
process_log = pd.DataFrame(procs)
process_log